In [1]:
!pip install mediapipe==0.10.14 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 13.8 MB/s eta 0:00:00


In [5]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.14


Found existing installation: mediapipe 0.10.14
Uninstalling mediapipe-0.10.14:
  Successfully uninstalled mediapipe-0.10.14
  Using cached mediapipe-0.10.14-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
Using cached mediapipe-0.10.14-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.


In [32]:
import os
print(os.getcwd())      # Shows current working directory
print(os.listdir())     # Lists files in the current directory


/content
['.config', 'input.MOV', 'mustache_filter_output.mp4', 'mustache.png', '.ipynb_checkpoints', 'sample_data']


In [30]:
import cv2
import numpy as np
import mediapipe as mp

# Load mustache image (must be RGBA)
mustache_img = cv2.imread('mustache.png', cv2.IMREAD_UNCHANGED)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Load video
cap = cv2.VideoCapture('/content/input.MOV')
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()
print(f"✅ Loaded {len(frames)} frames")

# Process each frame
output = []
for frame in frames:
    h, w = frame.shape[:2]
    result = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    if not result.multi_face_landmarks:
        output.append(frame)
        continue

    lm = result.multi_face_landmarks[0].landmark

    # Landmarks for nose and mouth area
    nose_tip = np.array([lm[1].x * w, lm[1].y * h])
    left_mouth = np.array([lm[61].x * w, lm[61].y * h])
    right_mouth = np.array([lm[291].x * w, lm[291].y * h])

    # Width and angle of mustache
    mustache_width = int(np.linalg.norm(left_mouth - right_mouth) * 1.8)  # Increased from 1.2 to 1.8
    mustache_height = int(mustache_width * mustache_img.shape[0] / mustache_img.shape[1])
    resized_overlay = cv2.resize(mustache_img, (mustache_width, mustache_height), interpolation=cv2.INTER_AREA)

    # Rotation angle
    angle_rad = np.arctan2(right_mouth[1] - left_mouth[1], right_mouth[0] - left_mouth[0])
    angle_deg = np.degrees(angle_rad)

    # Rotate mustache to allign with video
    (h_overlay, w_overlay) = resized_overlay.shape[:2]
    center = (w_overlay // 2, h_overlay // 2)
    M = cv2.getRotationMatrix2D(center, -angle_deg, 1.0)  # Negative for correct tilt
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h_overlay * sin) + (w_overlay * cos))
    new_h = int((h_overlay * cos) + (w_overlay * sin))
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    rotated_overlay = cv2.warpAffine(resized_overlay, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # Position: Center the mustache under the nose
    top_x = int(nose_tip[0] - rotated_overlay.shape[1] // 2 + 50)
    top_y = int(nose_tip[1] + mustache_height * 0.2 - 145)  # Offsets for x,y. Algorithm was a little off consistently by the same amount

    # Apply alpha blending to overlay mustache on the frame
    alpha = rotated_overlay[:, :, 3] / 255.0
    for c in range(3):
        frame[top_y:top_y + rotated_overlay.shape[0], top_x:top_x + rotated_overlay.shape[1], c] = \
            (1 - alpha) * frame[top_y:top_y + rotated_overlay.shape[0], top_x:top_x + rotated_overlay.shape[1], c] + \
            alpha * rotated_overlay[:, :, c]

    output.append(frame)

# Save output as MP4
if output:
    bgr_frames = [cv2.cvtColor(f, cv2.COLOR_RGB2BGR) for f in output]
    height, width = bgr_frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('mustache_filter_output.mp4', fourcc, 10.0, (width, height))
    for frame in bgr_frames:
        out.write(frame)
    out.release()
    print("✅ Saved: mustache_filter_output.mp4")
else:
    print("❌ No frames processed.")


✅ Loaded 123 frames
✅ Saved: mustache_filter_output.mp4
